This script contains the CRNN codes for Seo,B.(2024), 'Econometric Forecasting Using Ubiquitous News Texts: Text-enhanced Factor Model' International Journal of Forecasting

-Last Update: 2024-11-24 <br>
-Author: Beomseok Seo (bsseo@sookmyung.ac.kr)

### 0. <font color=purple> Environment Setup

In [41]:
import os, copy, pickle
from os import walk
from pathlib import Path
import pandas as pd
import numpy as np
import datetime as dt
from dateutil.relativedelta import relativedelta

In [42]:
import matplotlib.pyplot as plt
import seaborn as sns

In [43]:
import statsmodels.api as sm
import statsmodels as sm_

In [44]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [45]:
# To display pandas in full dataframe
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [46]:
# Set criteria
def rmse(y_pred,y_true):
    return np.sqrt(np.mean((y_pred-y_true)**2))
def mae(y_pred,y_true):
    return np.mean(np.abs(y_pred-y_true))

### 1. <font color='purple'>Load Dataset

In [47]:
TODAY = '240101'

In [48]:
all_dat = pd.read_csv('./data/all_dat_YoY.csv', index_col=0)
all_grp = pd.read_csv('./data/all_grp.csv', index_col=0)
macro_feat = pd.read_csv('./data/macro_feat.csv', index_col=0)

In [49]:
all_dat.index = pd.PeriodIndex(all_dat.index, freq='M')

In [50]:
all_grp_eng = copy.deepcopy(all_grp)

In [51]:
all_grp_eng_index = \
['GDP(SA)(Q)','GDP(NSA)(Q)',
 'Private consumption(SA)(Q)', 'Government consumption(SA)(Q)', 'Construction(SA)(Q)', 'Facility investment(SA)(Q)',
 'Exports of goods and services(SA)(Q)', 'Imports of goods and services(SA)(Q)',
 'Private consumption(NSA)(Q)', 'Government consumption(NSA)(Q)', 'Construction(NSA)(Q)', 'Facility investment(NSA)(Q)',
 'Exports of goods and services(NSA)(Q)', 'Imports of goods and services(NSA)(Q)',
 'Unemployment rate', 'Employment to population ratio', 'Number of employed people',
 'Monthly goods exports','Monthly goods imports',
 'Export price index','Import price index','Producer price index','Consumer price index',
 'Price index excluding agricultural product & oil', 'Price index excluding food & energy',
 'Consumption & Retail sales index(SA)','Service industry production index(SA)',
 'Consumption & Retail sales index(NSA)','Service industry production index(NSA)',
 'Manufacturing industry production index(SA)', 'Manufacturing industry shipment index(SA)', 'Manufacturing inventory index(SA)',
 'Manufacturing industry production index(NSA)', 'Manufacturing industry shipment index(NSA)', 'Manufacturing inventory index(NSA)',
 'Facility investment index(SA)','Construction completed(SA)',
 'Facility investment index(NSA)','Construction completed(NSA)',
 'Manufacturing business performance BSI(SA)','Manufacturing business performance BSI(NSA)',
 'All industries performance BSI', 'Service industry performance BSI', 'All industries sales BSI',
 'Manufacturing export BSI', 'Manufacturing domestic demand sales BSI', 'Manufacturing new orders BSI', 'Manufacturing operation rate BSI',
 'Economic sentiment index', 'Current economic judgment CSI', 'Consumer sentiment index',
 'Consolidated fiscal balance',
 'Housing sales price index(HSPI)-Seoul', 'Housing sales price index(HSPI)-National', 'Housing lease price index(HLPI)-Seoul', 'Housing lease price index(HLPI)-National',
 'Call rate','CD rate','KTB 3-year rate','KRW exchange rate', 'EUR exchange rate', 'KOSPI', 'KOSDAQ', 
 'Dubai crude oil', 'WTI futures', 'Gold futures',
 'Production(T)','Shipbuilding(T)','Automotive(T)','Semiconductor(T)','Facility investment(T)','Construction(T)',
 'Unemployment(T)','Recruitment(T)','Job search(T)',
 'Wholesale & retail(T)', 'Government expenditure(T)','Price outlook(T)','Stock price outlook(T)','House price outlook(T)',
 'World trade(T)',
 'News sentiment index(T)','Economic policy uncertainty(T)']

In [52]:
all_grp_eng.columns = ['External','Production','Labor','Consumption','Facility Inv.','Construction','Government','Exports','Imports','Prices','Finance','Real estate','Sentiments']

In [53]:
all_dat_eng = copy.deepcopy(all_dat)
all_dat_eng.columns = all_grp_eng_index
all_grp_eng.index = all_grp_eng_index

In [54]:
all_dat = all_dat_eng
all_grp = all_grp_eng

### 2. <font color='purple'>Generate vintage data

In [55]:
macro_lag = macro_feat['LAG'].values
finan_lag = np.zeros(14)
text_lag = np.zeros(17)

all_lag = np.concatenate([macro_lag,finan_lag,text_lag])

In [56]:
test_dates = [str(x) for x in all_dat.index if str(x)>='2016-01' and int(str(x)[-2:])%3==0][:-1]

In [57]:
def _GenVintage(all_dat):
    vintage_dict = dict()
    for i in test_dates:
        temp_dat = copy.deepcopy(all_dat)
        temp_dat = temp_dat.loc[:i]

        for j,l in enumerate(all_lag):
            if l==0:
                continue
            temp_dat.iloc[-int(l):,j] = np.nan

        vintage_dict[i] = temp_dat

    return vintage_dict

In [58]:
vintage_dict = _GenVintage(all_dat)

### 3. <font color='purple'>Experiments

#### convolutional recurrent neural network

In [69]:
factornames = {1:'External',2:'Production',3:'Labor',4:'Consumption',5:'Facility Inv.',6:'Construction',7:'Government',8:'Exports',9:'Imports',10:'Prices',11:'Finance',12:'Real estate',13:'Sentiments'}

In [70]:
factor_multiplicities = {'Global': 2}
factor_orders = {'Global': 4}

In [71]:
model_dict = dict()
results_dict = dict()

In [72]:
vintage_dict_keys = ['2016-03', '2016-06', '2016-09', '2016-12', '2017-03', '2017-06', '2017-09', '2017-12', '2018-03', '2018-06', '2018-09', '2018-12', '2019-03', '2019-06', '2019-09', '2019-12', '2020-03', '2020-06', '2020-09', '2020-12', '2021-03', '2021-06', '2021-09', '2021-12', '2022-03', '2022-06', '2022-09', '2022-12', '2023-03', '2023-06', '2023-09']

In [73]:
for fhor in ['now','1qa']:
    for trans in ['MoM','OS_MoM','NE_MoM','YoY','OS_YoY','NE_YoY']:

        all_dat_P = copy.deepcopy(all_dat)
        all_grp_P = copy.deepcopy(all_grp)

        if trans[:2] == 'NE':
            all_lag = np.concatenate([macro_lag[:14],text_lag])
            for i,x in enumerate(all_dat_P.columns):
                if x[-3:] != '(T)' and i>=14:
                    all_dat_P.drop(x,axis=1, inplace=True)
                    all_grp_P.drop(x,axis=0, inplace=True)

        elif trans[:2] == 'OS':
            all_lag = np.concatenate([macro_lag,finan_lag])
            for i,x in enumerate(all_dat_P.columns):
                if x[-3:] == '(T)':
                    all_dat_P.drop(x,axis=1, inplace=True)
                    all_grp_P.drop(x,axis=0, inplace=True)  
                    
        else:
            all_lag = np.concatenate([macro_lag,finan_lag,text_lag])

        vitage_dict = _GenVintage(all_dat_P)
        
        
        pred_crnn = []
        err_crnn = []
        true_crnn = []
        #==========================================================================================
        for v in vintage_dict_keys:
            vintage_dat = vintage_dict[v]

            if trans[-3:] in ['YoY']:
                item = 'GDP(NSA)(Q)'
            elif trans[-3:] in ['MoM']:
                item = 'GDP(SA)(Q)'

            if fhor == 'now':
                vd_train_x_m = vintage_dat.iloc[12:-3,14:].fillna(method='ffill').fillna(method='bfill')
                vd_train_x_q = vintage_dat.iloc[9:-6,:14].dropna()
                vd_train_y_q = vintage_dat.iloc[12:-3,:14][item].dropna()

                testing_x_m = np.expand_dims(vintage_dat.iloc[-36:,14:].fillna(method='ffill'),(0,3))
                testing_x_q = np.expand_dims(vintage_dat.iloc[-39:-3,:14].dropna(),0)
                testing_y_q = np.expand_dims(all_dat.loc[str(vintage_dat.iloc[[-1],:].index[0]), item],0)
            elif fhor == '1qa':
                vd_train_x_m = vintage_dat.iloc[9:-6,14:].fillna(method='ffill').fillna(method='bfill')
                vd_train_x_q = vintage_dat.iloc[6:-9,:14].dropna()
                vd_train_y_q = vintage_dat.iloc[12:-3,:14][item].dropna()

                testing_x_m = np.expand_dims(vintage_dat.iloc[-39:-3,14:].fillna(method='ffill'),(0,3))
                testing_x_q = np.expand_dims(vintage_dat.iloc[-42:-6,:14].dropna(),0)
                testing_y_q = np.expand_dims(all_dat.loc[str(vintage_dat.iloc[[-1],:].index[0]), item],0)

            ds_train_x_m = tf.keras.utils.timeseries_dataset_from_array(vd_train_x_m, targets=None, sequence_length=36, sequence_stride=3)
            ds_train_x_q = tf.keras.utils.timeseries_dataset_from_array(vd_train_x_q, None, sequence_length=12, sequence_stride=1)
            ds_train_y_q = tf.keras.utils.timeseries_dataset_from_array(vd_train_y_q, None, sequence_length=1, sequence_stride=1, start_index=11)


            train_x_m = np.transpose(np.stack(list(ds_train_x_m)),(1,2,3,0))
            train_x_q = np.squeeze(np.transpose(np.stack(list(ds_train_x_q)),(1,2,3,0)))
            train_y_q = np.squeeze(np.stack(list(ds_train_y_q)))

            print(np.sum(np.isnan(train_x_m)),np.sum(np.isnan(train_x_q)),np.sum(np.isnan(train_y_q)))
            print(train_x_m.shape, train_x_q.shape, train_y_q.shape)

            tf_train_x_mq = tf.data.Dataset.from_tensor_slices((train_x_m,train_x_q))
            tf_train_y_q = tf.data.Dataset.from_tensor_slices(train_y_q)

            train_ds = tf.data.Dataset.zip((tf_train_x_mq, tf_train_y_q)).batch(32).repeat()

            tf_testing_x_mq = tf.data.Dataset.from_tensor_slices((testing_x_m,testing_x_q))
            tf_testing_y_q = tf.data.Dataset.from_tensor_slices(testing_y_q)

            testing_ds = tf.data.Dataset.zip((tf_testing_x_mq, tf_testing_y_q)).batch(32).repeat()


            #---------------------------------------------------------------------------------------------------
            inp_m = tf.keras.layers.Input(shape=train_x_m.shape[1:])
            inp_q = tf.keras.layers.Input(shape=train_x_q.shape[1:])
            #---------------------------------------------------------------------------------------------------
            xm = tf.keras.layers.Conv2D(filters=64, kernel_size=(3,vd_train_x_m.shape[1]), strides=3, activation='relu', use_bias=True)(inp_m)
            xm = tf.keras.layers.Lambda(lambda x: x[:, :, 0, :])(xm)
            xmq = tf.keras.layers.Lambda(lambda x: tf.concat(x,axis=-1))([xm,inp_q])
            xx = tf.keras.layers.LSTM(32)(xmq)
            xx = tf.keras.layers.Dense(4, activation='relu', use_bias=True)(xx)
            out = tf.keras.layers.Dense(1, activation='linear', use_bias=True)(xx)
            
            #---------------------------------------------------------------------------------------------------
            model_lin = tf.keras.models.Model([inp_m,inp_q],out)
            #---------------------------------------------------------------------------------------------------

            EVALUATION_INTERVAL = 500
            EPOCHS = 5

            model_lin.compile(loss=tf.losses.MeanAbsoluteError(),
                        optimizer=tf.optimizers.Adam(learning_rate = 0.002),
                        metrics=[tf.metrics.MeanSquaredError()])

            history = model_lin.fit(train_ds,
                                steps_per_epoch=EVALUATION_INTERVAL,
                                epochs=EPOCHS)#, callbacks=[early_stopping])


            Path("./outs/crnn_out_{}_{}".format(trans,fhor)).mkdir(parents=True, exist_ok=True)
            model_lin.save('./outs/crnn_out_{}_{}/crnn_{}.keras'.format(trans,fhor,v))

            pred_nc = model_lin.predict((testing_x_m,testing_x_q))[0][0]

            pred_crnn.append(pred_nc)
            err_crnn.append(testing_y_q[0]-pred_nc)
            true_crnn.append(testing_y_q[0])
            print(v, 'pred:',pred_nc, 'true:',testing_y_q[0], 'err:',testing_y_q[0]-pred_nc)

        crnn_ = pd.DataFrame({'index':vintage_dict_keys, 'true':true_crnn, 'pred':pred_crnn,'err':err_crnn})
        crnn_.set_index('index', inplace=True)

#         Path("./outs/collect").mkdir(parents=True, exist_ok=True)
#         crnn_.to_csv('./outs/collect/crnn_{}_{}.csv'.format(trans,fhor))
#         print(fhor, trans, np.mean(np.abs(crnn_['err'])))            

0 0 0
(29, 36, 69, 1) (29, 12, 14) (29,)
1/1 [==============================] - 0s 388ms/step
2016-03 pred: -0.013270665 true: 0.3 err: 0.31327066496014594
0 0 0
(30, 36, 69, 1) (30, 12, 14) (30,)
1/1 [==============================] - 0s 178ms/step
2016-06 pred: 0.15496866 true: 1.2 err: 1.0450313359498977
0 0 0
(31, 36, 69, 1) (31, 12, 14) (31,)
1/1 [==============================] - 0s 175ms/step
2016-09 pred: 0.30977768 true: 0.4 err: 0.09022232294082644
0 0 0
(32, 36, 69, 1) (32, 12, 14) (32,)
1/1 [==============================] - 0s 176ms/step
2016-12 pred: 0.0019999933 true: 0.6 err: 0.5980000066570937
0 0 0
(33, 36, 69, 1) (33, 12, 14) (33,)
1/1 [==============================] - 0s 179ms/step
2017-03 pred: 0.10099488 true: 1.0 err: 0.8990051224827766
0 0 0
(34, 36, 69, 1) (34, 12, 14) (34,)
1/1 [==============================] - 0s 176ms/step
2017-06 pred: -0.021193665 true: 0.8 err: 0.821193664520979
0 0 0
(35, 36, 69, 1) (35, 12, 14) (35,)
1/1 [=============================

1/1 [==============================] - 0s 174ms/step
2016-06 pred: 1.0483643 true: 1.2 err: 0.15163571834564205
0 0 0
(31, 36, 69, 1) (31, 12, 14) (31,)
1/1 [==============================] - 0s 175ms/step
2016-09 pred: 0.047793277 true: 0.4 err: 0.35220672339200976
0 0 0
(32, 36, 69, 1) (32, 12, 14) (32,)
1/1 [==============================] - 0s 172ms/step
2016-12 pred: 0.7524314 true: 0.6 err: -0.15243139266967776
0 0 0
(33, 36, 69, 1) (33, 12, 14) (33,)
1/1 [==============================] - 0s 173ms/step
2017-03 pred: 0.0019999933 true: 1.0 err: 0.9980000066570938
0 0 0
(34, 36, 69, 1) (34, 12, 14) (34,)
1/1 [==============================] - 0s 172ms/step
2017-06 pred: 0.099976406 true: 0.8 err: 0.7000235944986344
0 0 0
(35, 36, 69, 1) (35, 12, 14) (35,)
1/1 [==============================] - 0s 171ms/step
2017-09 pred: 0.41137302 true: 1.4 err: 0.9886269807815551
0 0 0
(36, 36, 69, 1) (36, 12, 14) (36,)
1/1 [==============================] - 0s 172ms/step
2017-12 pred: -0.107438

1/1 [==============================] - 0s 172ms/step
2016-12 pred: 0.69343376 true: 0.6 err: -0.09343376159667971
0 0 0
(33, 36, 69, 1) (33, 12, 14) (33,)
1/1 [==============================] - 0s 172ms/step
2017-03 pred: 0.062829114 true: 1.0 err: 0.9371708855032921
0 0 0
(34, 36, 69, 1) (34, 12, 14) (34,)
1/1 [==============================] - 0s 174ms/step
2017-06 pred: 0.33583114 true: 0.8 err: 0.46416886448860173
0 0 0
(35, 36, 69, 1) (35, 12, 14) (35,)
1/1 [==============================] - 0s 173ms/step
2017-09 pred: 0.36518034 true: 1.4 err: 1.0348196566104888
0 0 0
(36, 36, 69, 1) (36, 12, 14) (36,)
1/1 [==============================] - 0s 177ms/step
2017-12 pred: 0.8839116 true: -0.3 err: -1.1839116096496582
0 0 0
(37, 36, 69, 1) (37, 12, 14) (37,)
1/1 [==============================] - 0s 175ms/step
2018-03 pred: 0.030591222 true: 1.3 err: 1.269408778473735
0 0 0
(38, 36, 69, 1) (38, 12, 14) (38,)
1/1 [==============================] - 0s 172ms/step
2018-06 pred: -0.0604084

1/1 [==============================] - 0s 180ms/step
2017-06 pred: 0.54586536 true: 2.8 err: 2.25413464307785
0 0 0
(35, 36, 69, 1) (35, 12, 14) (35,)
1/1 [==============================] - 0s 175ms/step
2017-09 pred: 0.7026167 true: 3.9 err: 3.1973833084106444
0 0 0
(36, 36, 69, 1) (36, 12, 14) (36,)
1/1 [==============================] - 0s 172ms/step
2017-12 pred: 0.07996242 true: 2.9 err: 2.82003758251667
0 0 0
(37, 36, 69, 1) (37, 12, 14) (37,)
1/1 [==============================] - 0s 179ms/step
2018-03 pred: 0.3849195 true: 2.9 err: 2.5150805056095122
0 0 0
(38, 36, 69, 1) (38, 12, 14) (38,)
1/1 [==============================] - 0s 173ms/step
2018-06 pred: -0.20722735 true: 3.1 err: 3.307227349281311
0 0 0
(39, 36, 69, 1) (39, 12, 14) (39,)
1/1 [==============================] - 0s 174ms/step
2018-09 pred: 0.7528165 true: 2.4 err: 1.6471835017204284
0 0 0
(40, 36, 69, 1) (40, 12, 14) (40,)
1/1 [==============================] - 0s 175ms/step
2018-12 pred: 0.15108603 true: 3.2 e

1/1 [==============================] - 0s 174ms/step
2017-12 pred: 0.096389964 true: 2.9 err: 2.803610035777092
0 0 0
(37, 36, 69, 1) (37, 12, 14) (37,)
1/1 [==============================] - 0s 173ms/step
2018-03 pred: 0.68032885 true: 2.9 err: 2.2196711540222167
0 0 0
(38, 36, 69, 1) (38, 12, 14) (38,)
1/1 [==============================] - 0s 185ms/step
2018-06 pred: 0.080676734 true: 3.1 err: 3.019323265552521
0 0 0
(39, 36, 69, 1) (39, 12, 14) (39,)
1/1 [==============================] - 0s 178ms/step
2018-09 pred: 0.033500798 true: 2.4 err: 2.366499201953411
0 0 0
(40, 36, 69, 1) (40, 12, 14) (40,)
1/1 [==============================] - 0s 198ms/step
2018-12 pred: 0.005785026 true: 3.2 err: 3.1942149738781156
0 0 0
(41, 36, 69, 1) (41, 12, 14) (41,)
1/1 [==============================] - 0s 177ms/step
2019-03 pred: -0.091432974 true: 1.9 err: 1.991432973742485
0 0 0
(42, 36, 69, 1) (42, 12, 14) (42,)
1/1 [==============================] - 0s 183ms/step
2019-06 pred: -0.4672089 tr

...

1/1 [==============================] - 0s 195ms/step
2019-06 pred: -0.034779225 true: 1.1 err: 1.1347792245447637
0 0 0
(43, 36, 69, 1) (43, 12, 14) (43,)
1/1 [==============================] - 0s 184ms/step
2019-09 pred: 0.81437093 true: 0.4 err: -0.4143709301948547
0 0 0
(44, 36, 69, 1) (44, 12, 14) (44,)
1/1 [==============================] - 0s 186ms/step
2019-12 pred: 0.12538876 true: 1.2 err: 1.0746112436056137
0 0 0
(45, 36, 69, 1) (45, 12, 14) (45,)
1/1 [==============================] - 0s 190ms/step
2020-03 pred: 0.0003579238 true: -1.3 err: -1.3003579237964005
0 0 0
(46, 36, 69, 1) (46, 12, 14) (46,)
1/1 [==============================] - 0s 190ms/step
2020-06 pred: 0.48230204 true: -3.0 err: -3.482302039861679
0 0 0
(47, 36, 69, 1) (47, 12, 14) (47,)
1/1 [==============================] - 0s 188ms/step
2020-09 pred: -0.044810295 true: 2.3 err: 2.3448102951049803
0 0 0
(48, 36, 69, 1) (48, 12, 14) (48,)
1/1 [==============================] - 0s 175ms/step
2020-12 pred: 0.080

1/1 [==============================] - 0s 174ms/step
2019-12 pred: 0.4642299 true: 1.2 err: 0.7357700884342193
0 0 0
(45, 36, 69, 1) (45, 12, 14) (45,)
1/1 [==============================] - 0s 179ms/step
2020-03 pred: -0.0028663105 true: -1.3 err: -1.297133689466864
0 0 0
(46, 36, 69, 1) (46, 12, 14) (46,)
1/1 [==============================] - 0s 178ms/step
2020-06 pred: 0.41335833 true: -3.0 err: -3.4133583307266235
0 0 0
(47, 36, 69, 1) (47, 12, 14) (47,)
1/1 [==============================] - 0s 182ms/step
2020-09 pred: 1.065223 true: 2.3 err: 1.2347770214080809
0 0 0
(48, 36, 69, 1) (48, 12, 14) (48,)
1/1 [==============================] - 0s 175ms/step
2020-12 pred: 0.25795394 true: 1.3 err: 1.042046058177948
0 0 0
(49, 36, 69, 1) (49, 12, 14) (49,)
1/1 [==============================] - 0s 176ms/step
2021-03 pred: 0.040703572 true: 1.8 err: 1.759296427667141
0 0 0
(50, 36, 69, 1) (50, 12, 14) (50,)
1/1 [==============================] - 0s 181ms/step
2021-06 pred: -0.05593886 t

1/1 [==============================] - 0s 174ms/step
2020-06 pred: 0.36447832 true: -2.6 err: -2.9644783198833466
0 0 0
(47, 36, 69, 1) (47, 12, 14) (47,)
1/1 [==============================] - 0s 175ms/step
2020-09 pred: 0.09068747 true: -0.9 err: -0.9906874686479569
0 0 0
(48, 36, 69, 1) (48, 12, 14) (48,)
1/1 [==============================] - 0s 172ms/step
2020-12 pred: 0.49345428 true: -0.7 err: -1.1934542775154113
0 0 0
(49, 36, 69, 1) (49, 12, 14) (49,)
1/1 [==============================] - 0s 173ms/step
2021-03 pred: 0.5873888 true: 2.4 err: 1.812611186504364
0 0 0
(50, 36, 69, 1) (50, 12, 14) (50,)
1/1 [==============================] - 0s 180ms/step
2021-06 pred: 0.25871482 true: 6.4 err: 6.141285175085068
0 0 0
(51, 36, 69, 1) (51, 12, 14) (51,)
1/1 [==============================] - 0s 172ms/step
2021-09 pred: 0.12695386 true: 4.1 err: 3.973046144843101
0 0 0
(52, 36, 69, 1) (52, 12, 14) (52,)
1/1 [==============================] - 0s 190ms/step
2021-12 pred: 0.0019999922 

1/1 [==============================] - 0s 177ms/step
2020-12 pred: -0.043534696 true: -0.7 err: -0.6564653038978576
0 0 0
(49, 36, 69, 1) (49, 12, 14) (49,)
1/1 [==============================] - 0s 178ms/step
2021-03 pred: 0.06622946 true: 2.4 err: 2.3337705373764037
0 0 0
(50, 36, 69, 1) (50, 12, 14) (50,)
1/1 [==============================] - 0s 179ms/step
2021-06 pred: -0.41723567 true: 6.4 err: 6.817235672473908
0 0 0
(51, 36, 69, 1) (51, 12, 14) (51,)
1/1 [==============================] - 0s 179ms/step
2021-09 pred: 0.09806809 true: 4.1 err: 4.001931911706924
0 0 0
(52, 36, 69, 1) (52, 12, 14) (52,)
1/1 [==============================] - 0s 181ms/step
2021-12 pred: 0.38618597 true: 4.3 err: 3.9138140261173247
0 0 0
(53, 36, 69, 1) (53, 12, 14) (53,)
1/1 [==============================] - 0s 177ms/step
2022-03 pred: 0.27615327 true: 3.1 err: 2.823846733570099
0 0 0
(54, 36, 69, 1) (54, 12, 14) (54,)
1/1 [==============================] - 0s 174ms/step
2022-06 pred: 0.06449944 tr

1/1 [==============================] - 0s 176ms/step
2021-06 pred: 0.15603505 true: 6.4 err: 6.243964949250222
0 0 0
(51, 36, 69, 1) (51, 12, 14) (51,)
1/1 [==============================] - 0s 175ms/step
2021-09 pred: -0.23601113 true: 4.1 err: 4.336011132597923
0 0 0
(52, 36, 69, 1) (52, 12, 14) (52,)
1/1 [==============================] - 0s 173ms/step
2021-12 pred: 0.31211868 true: 4.3 err: 3.9878813207149504
0 0 0
(53, 36, 69, 1) (53, 12, 14) (53,)
1/1 [==============================] - 0s 176ms/step
2022-03 pred: -0.15029047 true: 3.1 err: 3.2502904742956162
0 0 0
(54, 36, 69, 1) (54, 12, 14) (54,)
1/1 [==============================] - 0s 176ms/step
2022-06 pred: -0.108833745 true: 2.9 err: 3.008833745121956
0 0 0
(55, 36, 69, 1) (55, 12, 14) (55,)
1/1 [==============================] - 0s 231ms/step
2022-09 pred: -0.39637652 true: 3.2 err: 3.596376520395279
0 0 0
(56, 36, 69, 1) (56, 12, 14) (56,)
1/1 [==============================] - 0s 191ms/step
2022-12 pred: 0.41586566 tru